<a href="https://colab.research.google.com/github/pauloricardofagundes/MVP-Machine-Learning-e-Analytics/blob/main/MVP_MachineLearning%26Analytics_CartCredito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MVP - Machine Learning & Analytics**

## 1. O Projeto

O objetivo deste projeto será treinar um modelo de machine learning, que seja capaz de atribuir risco de fraude de acordo com os perfis analisados.

O dataset escolhido foi um relatório de transações por cartão de crédito de uma adminstradora americana, no primeiro semestre de 2019.

Devido ao tamanho do arquivo original, foi necessário dividi-lo, salvar os conjuntos no respositório do github e refazer essa união dos dados via python com um loop para leitura dos datasets.

Este dataset possui 25 campos, porém serão utilizados os 9 mais importantes pra essa análise e seus atributos são:

1. trans_date_trans_time:	Data e hora da transação
2. category:	Categoria do Estabelecimento
3. first:	Primeiro Nome
4. gender:	Genero
5. city:	Cidade
6. state:	Estado
7. job:	Profissão
8. dob:	Data de Nascimento
9. is_fraud:	Classificação de fraude


## 2. Importando o Dataset e verificando os dados


### 2.1 Importando Bibliotecas

In [1]:
## Carregamento de Bibliotecas

    # Manipula data frame
import pandas as pd
import numpy as np
import calendar
from datetime import datetime, date
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate

### 2.2 Importando o Dataset e verificando os dados

In [2]:
## Lista de datasets a serem utilizados para consolidar o arquivo final
    # Dados se referem a uso de cartão de crédito durante o primeiro semestre de 2019
Databases = {'https://raw.githubusercontent.com/pauloricardofagundes/MVP-Machine-Learning-e-Analytics/main/credit_card_transactions_1.csv',
             'https://raw.githubusercontent.com/pauloricardofagundes/MVP-Machine-Learning-e-Analytics/main/credit_card_transactions_2.csv',
             'https://raw.githubusercontent.com/pauloricardofagundes/MVP-Machine-Learning-e-Analytics/main/credit_card_transactions_3.csv',
             'https://raw.githubusercontent.com/pauloricardofagundes/MVP-Machine-Learning-e-Analytics/main/credit_card_transactions_4.csv'}

## Leitura dos datasets da lista e consolida todos em um único dataset
df_list = []

Campos_uteis = ['trans_date_trans_time', 'category', 'first', 'gender', 'city', 'state', 'job', 'dob', 'is_fraud']

for filename in sorted(Databases):
    df_list.append(pd.read_csv(filename,header=None, low_memory=False).set_index(0))
full_df = pd.concat(df_list)

## Remove linha com cabeçalho numérico criado pela função append
full_df = full_df.rename(columns=full_df.iloc[0]).drop(full_df.index[0])

## Filtrando colunas que serão utilizadas nas análises
full_df = full_df[Campos_uteis]

In [3]:
## Conferindo valores nulos das colunas
full_df.isnull().sum()

,0
trans_date_trans_time,0
category,0
first,0
gender,0
city,0
state,0
job,0
dob,0
is_fraud,0


In [4]:
## Resumo estatístico do Dataset final
full_df.describe()

,trans_date_trans_time,category,first,gender,city,state,job,dob,is_fraud
count,371999,371999,371999,371999,371999,371999,371999,371999,371999
unique,366513,14,341,2,859,51,481,922,4
top,2019-04-22 16:02:01,gas_transport,Christopher,F,Birmingham,TX,Film/video editor,1977-03-23,0
freq,4,37860,7825,203512,1616,27154,2794,1576,277387


In [5]:
## Conferindo os tipos de cada atributo
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 371999 entries, 0 to 371998
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   trans_date_trans_time  371999 non-null  object
 1   category               371999 non-null  object
 2   first                  371999 non-null  object
 3   gender                 371999 non-null  object
 4   city                   371999 non-null  object
 5   state                  371999 non-null  object
 6   job                    371999 non-null  object
 7   dob                    371999 non-null  object
 8   is_fraud               371999 non-null  object
dtypes: object(9)
memory usage: 28.4+ MB


Ao final deste primeiro processo, observo que neste database não temos valores nulos nas colunas que serão utilizadas. Todos os campos estão no formato object e os campos numéricos poderão ser convertidos conforme a necessidade de cada utilização.

## 3. Transformando os Dados

Transformando os campos de data (Data de Transação e Data de Nascimento) e demais valores em formatos que serão úteis nos agrupamentos necessários para as análises

In [14]:
## Separação de campos para Mês e Hora da transação
full_df['trans_date_trans_time'] = pd.to_datetime(full_df['trans_date_trans_time'], format="%Y-%m-%d %H:%M:%S", dayfirst=True)
full_df['Mes'] = full_df['trans_date_trans_time'].dt.strftime('%b/%y')
full_df['Hora'] = full_df['trans_date_trans_time'].dt.strftime('%H')

## Cálculo da Idade por meio do campo de data de nascimento
hoje = date.today()
full_df["Idade"] = (pd.Timestamp(hoje)
                 - pd.to_datetime(full_df["dob"], format="%Y-%m-%d", dayfirst=True)
                  ) // pd.Timedelta(days=365.2425)

## Função para atriuir faixa etária
def atribui_aging(age):
    if 0 <= age <= 25:
        return '0-25'
    elif 26 <= age <= 35:
        return '26-35'
    elif 36 <= age <= 45:
        return '36-45'
    elif 46 <= age <= 55:
        return '46-55'
    elif 55 <= age <= 65:
        return '56-65'
    elif 66 <= age <= 75:
        return '66-75'
    else:
        return '>75'

## Cria nova coluna com a faixa etaria da idade
full_df['Aging'] = full_df['Idade'].apply(atribui_aging)
## Cria nova coluna com o dia da semana da transação e substitui os nomes em inglês para abreviação em português
full_df['d_semana'] = full_df['trans_date_trans_time'].dt.day_name()
full_df['d_semana'] = np.where(full_df['d_semana'] == "Sunday", "Dom",
                               np.where(full_df['d_semana'] == "Monday", "Seg",
                                        np.where(full_df['d_semana'] == "Tuesday", "Ter",
                                                 np.where(full_df['d_semana'] == "Wednesday", "Qua",
                                                          np.where(full_df['d_semana'] == "Thursday", "Qui",
                                                                   np.where(full_df['d_semana'] == "Friday", "Sex",
                                                                            np.where(full_df['d_semana'] == "Saturday", "Sáb","-")))))))

## Conversão do campo 'Idade' para integer
full_df['Idade'] = pd.to_numeric(full_df["Idade"], errors='coerce', downcast="integer")
## Conversão do campo 'is_fraud' (binário para fraude) de string para integer
full_df['is_fraud'] = pd.to_numeric(full_df["is_fraud"], errors='coerce', downcast="integer")

#print(full_df)

## Obtendo a proporção de valores 0 e 1 na coluna binária "is_fraud"
valor_zero =round(((full_df['is_fraud'] == 0).sum() / len(full_df)) * 100,1)
valor_um = round(((full_df['is_fraud'] == 1).sum() / len(full_df)) * 100,1)

print(f"Percentual de 0's: {valor_zero}%")
print(f"Percentual de 1's: {valor_um}%")

## Separando o dataset entre falso e verdadeiro
full_df1 = full_df[full_df['is_fraud'] == 0]
full_df2 = full_df[full_df['is_fraud'] == 1]

## Criando um dataset reduzido em 2.500 registros, com base na proporção de Falso e Verdadeiro, pra otimizar o desempenho do processamento
Qtd_Zeros = int(valor_zero*25)
Qtd_Ums = int(valor_um*25)

full_dfF = full_df1.head(Qtd_Zeros)
full_dfV = full_df2.head(Qtd_Ums)

full_df_final = pd.concat([full_dfF, full_dfV])

print(full_df_final)
full_df_final.describe()

Percentual de 0's: 99.3%
Percentual de 1's: 0.7%
     trans_date_trans_time       category      first gender            city  \
0                                                                             
0      2019-01-01 00:00:18       misc_net   Jennifer      F  Moravian Falls   
1      2019-01-01 00:00:44    grocery_pos  Stephanie      F          Orient   
2      2019-01-01 00:00:51  entertainment     Edward      M      Malad City   
3      2019-01-01 00:01:16  gas_transport     Jeremy      M         Boulder   
4      2019-01-01 00:03:06       misc_pos      Tyler      M        Doe Hill   
...                    ...            ...        ...    ...             ...   
4644   2019-01-03 22:09:00       misc_net    Heather      F           Manor   
4654   2019-01-03 22:21:15   shopping_net      Jason      M   Collettsville   
4693   2019-01-03 22:58:44         travel    Heather      F           Manor   
4706   2019-01-03 23:09:51   shopping_pos      Jason      M   Collettsville   
472

,trans_date_trans_time,is_fraud,Idade
count,2499,2499.000000,2499.000000
mean,2019-01-01 13:54:35.888355328,0.006803,51.247299
min,2019-01-01 00:00:18,0.000000,19.000000
25%,2019-01-01 08:08:47.500000,0.000000,38.000000
50%,2019-01-01 14:42:54,0.000000,49.000000
75%,2019-01-01 19:30:00.500000,0.000000,62.000000
max,2019-01-03 23:29:57,1.000000,98.000000
std,NaN,0.082214,17.430672


Neste processo, foi necessário resumir o dataset, simulando um novo dataset para que a próxima etapa pudesse ter uma performance de excução otimizada.

## 4. Construção do Modelo de Treinamento e Teste

Nesta seção, será apresentado o modelo de treinamento e teste.

In [15]:
# Criação da coluna 'target' (assumindo que 'is_fraud' já representa o target)
full_df_final['target'] = full_df_final['is_fraud']

# Seleciona os atributos e o target
X = full_df_final.drop(['target','trans_date_trans_time', 'dob', 'is_fraud'], axis=1)
y = full_df_final['target']

# Converte variáveis categóricas para numéricas usando one-hot encoding
X = pd.get_dummies(X, columns=['category', 'first', 'gender', 'city', 'state', 'job', 'Mes', 'Hora', 'Aging', 'd_semana'])

# Define os modelos a serem avaliados
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Define os parâmetros para o GridSearchCV
param_grid = {
    'LR': {'LR__C': [0.1, 1, 10]},
    'KNN': {'KNN__n_neighbors': [3, 5, 7]},
    'CART': {'CART__max_depth': [3, 5, 7]},
    'SVM': {'SVM__C': [0.1, 1, 10]}
}

# Configura a validação cruzada
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Loop pelos modelos
results = []
names = []

for name, model in models:
    # Cria o pipeline com StandardScaler
    pipeline = Pipeline([('Scaler', StandardScaler()), (name, model)])

    # Executa o GridSearchCV
    if name in param_grid:
      grid = GridSearchCV(pipeline, param_grid[name], cv=kfold, scoring='accuracy')
      grid.fit(X, y)
      results.append(grid.best_score_)
      names.append(name)
      print(f"Melhor pontuação para {name}: {grid.best_score_} com parâmetros: {grid.best_params_}")
    else:
      cv_results = cross_val_score(pipeline, X, y, cv=kfold, scoring='accuracy')
      results.append(cv_results)
      names.append(name)
      print(f"Resultados da validação cruzada para {name}: {cv_results}")
      print(f"Média de Acurácia para {name}: {cv_results.mean()}")

Melhor pontuação para LR: 0.9995991983967937 com parâmetros: {'LR__C': 0.1}
Melhor pontuação para KNN: 0.9971975951903808 com parâmetros: {'KNN__n_neighbors': 7}
Melhor pontuação para CART: 0.9991991983967937 com parâmetros: {'CART__max_depth': 3}
Resultados da validação cruzada para NB: [0.998      1.         1.         1.         0.99799599]
Média de Acurácia para NB: 0.9991991983967937
Melhor pontuação para SVM: 0.9995991983967937 com parâmetros: {'SVM__C': 10}


## 5. Conclusão

Dos modelos avaliados, todos apresentaram resultados excelentes.
A acurácia foi considerara satisfatória com resultado de 0,99.

---

